In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN, GRU, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer



Question 1

Reading the folder and all its files

In [ ]:
import os

# Path to the dataset folder
dataset_path = "D:\\NLP\\data"

# List files and directories in the dataset folder
print(os.listdir(dataset_path))

# Accessing train and test data
train_path = os.path.join(dataset_path, "train")
test_path = os.path.join(dataset_path, "test")

# List files and directories in the train and test folders
print("Train folder contents:", os.listdir(train_path))
print("Test folder contents:", os.listdir(test_path))

# Accessing train data subfolders (neg, pos, unsup)
train_neg_path = os.path.join(train_path, "neg")
train_pos_path = os.path.join(train_path, "pos")
train_unsup_path = os.path.join(train_path, "unsup")

# List files in the train subfolders
print("Train neg folder contents:", os.listdir(train_neg_path))
print("Train pos folder contents:", os.listdir(train_pos_path))
print("Train unsup folder contents:", os.listdir(train_unsup_path))

# Accessing test data subfolders (neg, pos)
test_neg_path = os.path.join(test_path, "neg")
test_pos_path = os.path.join(test_path, "pos")

# List files in the test subfolders
print("Test neg folder contents:", os.listdir(test_neg_path))
print("Test pos folder contents:", os.listdir(test_pos_path))


['imdb.vocab', 'imdbEr.txt', 'README', 'test', 'train']
Train folder contents: ['labeledBow.feat', 'neg', 'pos', 'unsup', 'unsupBow.feat', 'urls_neg.txt', 'urls_pos.txt', 'urls_unsup.txt']
Test folder contents: ['labeledBow.feat', 'neg', 'pos', 'urls_neg.txt', 'urls_pos.txt']
Train neg folder contents: ['0_3.txt', '10000_4.txt', '10001_4.txt', '10002_1.txt', '10003_1.txt', '10004_3.txt', '10005_3.txt', '10006_4.txt', '10007_1.txt', '10008_2.txt', '10009_1.txt', '1000_4.txt', '10010_3.txt', '10011_3.txt', '10012_1.txt', '10013_1.txt', '10014_2.txt', '10015_2.txt', '10016_4.txt', '10017_4.txt', '10018_3.txt', '10019_3.txt', '1001_4.txt', '10020_3.txt', '10021_2.txt', '10022_4.txt', '10023_1.txt', '10024_3.txt', '10025_1.txt', '10026_2.txt', '10027_1.txt', '10028_2.txt', '10029_1.txt', '1002_3.txt', '10030_1.txt', '10031_2.txt', '10032_4.txt', '10033_1.txt', '10034_1.txt', '10035_1.txt', '10036_1.txt', '10037_1.txt', '10038_3.txt', '10039_1.txt', '1003_3.txt', '10040_2.txt', '10041_1.txt'

Making DataFrame

In [ ]:
import pandas as pd
import os

# Function to load text data from a folder
def load_text_data(folder_path,limit=2500):
    texts = []
    labels = []
    for file in os.listdir(folder_path):
        with open(os.path.join(folder_path, file), 'r', encoding='utf-8') as f:
            texts.append(f.read())
            if 'pos' in folder_path:
                labels.append(1)
            elif 'neg' in folder_path:
                labels.append(0)
    return texts, labels

# Load train data
train_neg_texts, train_neg_labels = load_text_data(train_neg_path)
train_pos_texts, train_pos_labels = load_text_data(train_pos_path)
train_texts = train_neg_texts + train_pos_texts
train_labels = train_neg_labels + train_pos_labels

# Load test data
test_neg_texts, test_neg_labels = load_text_data(test_neg_path)
test_pos_texts, test_pos_labels = load_text_data(test_pos_path)
test_texts = test_neg_texts + test_pos_texts
test_labels = test_neg_labels + test_pos_labels

# Convert the text data into a DataFrame
train_df = pd.DataFrame({'text': train_texts, 'label': train_labels})
test_df = pd.DataFrame({'text': test_texts, 'label': test_labels})

# Shuffle the data
train_df = train_df.sample(frac=1).reset_index(drop=True)
test_df = test_df.sample(frac=1).reset_index(drop=True)

# Display the first few rows of the train and test DataFrames
print("Train data:")
print(train_df.head())
print("Test data:")
print(test_df.head())


Train data:
                                                text  label
0  This movie was 100% boring, i swear i almost d...      0
1  The first Cube movie was an art movie. It set ...      0
2  Tenchu aka. Hitokiri- directed by Hideo Gosha ...      1
3  I only watched this because it starred Josie L...      1
4  I had no idea what this movie was until I read...      1
Test data:
                                                text  label
0  This oddity contains Bunuel-like touches, but ...      0
1  kite runner is undoubtedly one of the most ama...      1
2  The cast was well picked. Pauly Shore is hilar...      1
3  If this is what's best in the Finnish cinema a...      0
4  Yes, I felt like I had been gutted after first...      1


Bag of words

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Initialize the CountVectorizer
vectorizer = CountVectorizer()

# Fit and transform the training data
X_train_bow = vectorizer.fit_transform(train_df['text'])

# Transform the testing data
X_test_bow = vectorizer.transform(test_df['text'])

# Display the shape of the BoW vectors
print("Shape of BoW vectors (training):", X_train_bow.shape)
print("Shape of BoW vectors (testing):", X_test_bow.shape)


Shape of BoW vectors (training): (25000, 74849)
Shape of BoW vectors (testing): (25000, 74849)


RNN

In [ ]:
# Define the batch size
batch_size = 1000

max_seq_length = 100  # Assuming maximum sequence length of 100 for padding

# Calculate the number of batches
num_batches_train = X_train_bow.shape[0] // batch_size
num_batches_test = X_test_bow.shape[0] // batch_size


# Initialize empty lists to store padded sequences
X_train_pad_batches = []
X_test_pad_batches = []

# Process training data in batches
for i in range(num_batches_train):
    start_index = i * batch_size
    end_index = min((i + 1) * batch_size, X_train_bow.shape[0])
    X_train_pad_batch = pad_sequences(X_train_bow[start_index:end_index].toarray(), maxlen=max_seq_length)
    X_train_pad_batches.append(X_train_pad_batch)

# Process the last batch of training data (if any)
if X_train_bow.shape[0] % batch_size != 0:
    start_index = num_batches_train * batch_size
    X_train_pad_batch = pad_sequences(X_train_bow[start_index:].toarray(), maxlen=max_seq_length)
    X_train_pad_batches.append(X_train_pad_batch)

# Process testing data in batches
for i in range(num_batches_test):
    start_index = i * batch_size
    end_index = min((i + 1) * batch_size, X_test_bow.shape[0])
    X_test_pad_batch = pad_sequences(X_test_bow[start_index:end_index].toarray(), maxlen=max_seq_length)
    X_test_pad_batches.append(X_test_pad_batch)

# Process the last batch of testing data (if any)
if X_test_bow.shape[0] % batch_size != 0:
    start_index = num_batches_test * batch_size
    X_test_pad_batch = pad_sequences(X_test_bow[start_index:].toarray(), maxlen=max_seq_length)
    X_test_pad_batches.append(X_test_pad_batch)


# Concatenate batches into single arrays
X_train_pad = np.concatenate(X_train_pad_batches)
X_test_pad = np.concatenate(X_test_pad_batches)


In [ ]:
X_train_pad_shortened = X_train_pad[:1000]
y_train_shortened = y_train[:1000]


In [ ]:
X_test_pad_shortened = X_test_pad[:1000]
y_test_shortened = y_test[:1000]


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Define the RNN model
model_rnn = Sequential()
model_rnn.add(Embedding(input_dim=len(vectorizer.vocabulary_), output_dim=100, input_length=max_seq_length))
model_rnn.add(SimpleRNN(units=64, activation='relu'))
model_rnn.add(Dense(1, activation='sigmoid'))

# Compile the model
model_rnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model with shortened data
model_rnn.fit(X_train_pad_shortened, y_train_shortened, epochs=5, batch_size=64, validation_split=0.2)

# Evaluate the model on the shortened test set
scores = model_rnn.evaluate(X_test_pad_shortened, y_test_shortened)
print("Test Accuracy:", scores[1])

# Predict probabilities for the shortened test set
y_pred_prob = model_rnn.predict(X_test_pad_shortened)

# Convert probabilities to class labels
y_pred = (y_pred_prob > 0.5).astype('int32')

# Calculate accuracy
accuracy = accuracy_score(y_test_shortened, y_pred)
print("Accuracy:", accuracy)

# Calculate precision, recall, and F1-score
precision = precision_score(y_test_shortened, y_pred)
recall = recall_score(y_test_shortened, y_pred)
f1 = f1_score(y_test_shortened, y_pred)

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


Epoch 1/5


c:\Users\Lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\embedding.py:86: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 142ms/step - accuracy: 1.0000 - loss: 0.4550 - val_accuracy: 1.0000 - val_loss: 1.6647e-05
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 125ms/step - accuracy: 1.0000 - loss: 3.8827e-06 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 127ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 125ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 127ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 1.0000 - loss: 0.0000e+00
Test Accuracy: 1.0
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step
Accuracy: 1.0
Precision: 0.0
Recall: 0.0
F1 Score: 0.0


c:\Users\Lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1757: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


GRU

In [ ]:
from keras.layers import GRU

# Define the GRU model
model_gru = Sequential()
model_gru.add(Embedding(input_dim=len(vectorizer.vocabulary_), output_dim=100, input_length=max_seq_length))
model_gru.add(GRU(units=64, activation='relu'))
model_gru.add(Dense(1, activation='sigmoid'))

# Compile the model
model_gru.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model with shortened data
model_gru.fit(X_train_pad_shortened, y_train_shortened, epochs=5, batch_size=64, validation_split=0.2)

# Evaluate the model on the shortened test set
scores = model_gru.evaluate(X_test_pad_shortened, y_test_shortened)
print("Test Accuracy (GRU):", scores[1])


Epoch 1/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 174ms/step - accuracy: 1.0000 - loss: 0.5780 - val_accuracy: 1.0000 - val_loss: 0.2097
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 153ms/step - accuracy: 1.0000 - loss: 0.1304 - val_accuracy: 1.0000 - val_loss: 0.0060
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 158ms/step - accuracy: 1.0000 - loss: 0.0027 - val_accuracy: 1.0000 - val_loss: 1.6425e-05
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 146ms/step - accuracy: 1.0000 - loss: 7.7322e-06 - val_accuracy: 1.0000 - val_loss: 3.2809e-07
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 148ms/step - accuracy: 1.0000 - loss: 2.2159e-07 - val_accuracy: 1.0000 - val_loss: 7.5765e-08
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 1.0000 - loss: 7.7574e-08
Test Accuracy (GRU): 1.0


LSTM

In [ ]:
from keras.layers import LSTM

# Define the LSTM model
model_lstm = Sequential()
model_lstm.add(Embedding(input_dim=len(vectorizer.vocabulary_), output_dim=100, input_length=max_seq_length))
model_lstm.add(LSTM(units=64, activation='relu'))
model_lstm.add(Dense(1, activation='sigmoid'))

# Compile the model
model_lstm.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model with shortened data
model_lstm.fit(X_train_pad_shortened, y_train_shortened, epochs=5, batch_size=64, validation_split=0.2)

# Evaluate the model on the shortened test set
scores = model_lstm.evaluate(X_test_pad_shortened, y_test_shortened)
print("Test Accuracy (LSTM):", scores[1])


Epoch 1/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 157ms/step - accuracy: 0.7498 - loss: 0.5924 - val_accuracy: 1.0000 - val_loss: 0.0558
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 142ms/step - accuracy: 1.0000 - loss: 0.1891 - val_accuracy: 1.0000 - val_loss: 0.4392
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 143ms/step - accuracy: 1.0000 - loss: 0.4382 - val_accuracy: 1.0000 - val_loss: 0.3496
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 140ms/step - accuracy: 1.0000 - loss: 0.1879 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 140ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 1.0000 - loss: 0.0000e+00
Test Accuracy (LSTM): 1.0


BiLSTM

In [ ]:
from keras.layers import Bidirectional

# Define the BiLSTM model
model_bilstm = Sequential()
model_bilstm.add(Embedding(input_dim=len(vectorizer.vocabulary_), output_dim=100, input_length=max_seq_length))
model_bilstm.add(Bidirectional(LSTM(units=64, activation='relu')))
model_bilstm.add(Dense(1, activation='sigmoid'))

# Compile the model
model_bilstm.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model with shortened data
model_bilstm.fit(X_train_pad_shortened, y_train_shortened, epochs=5, batch_size=64, validation_split=0.2)

# Evaluate the model on the shortened test set
scores = model_bilstm.evaluate(X_test_pad_shortened, y_test_shortened)
print("Test Accuracy (BiLSTM):", scores[1])


Epoch 1/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 201ms/step - accuracy: 1.0000 - loss: 0.4949 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 160ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 160ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 165ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 161ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 1.0000 - loss: 0.0000e+00
Test Accuracy (BiLSTM): 1.0


In [ ]:
# Reshape input data for RNN, GRU, LSTM, or BiLSTM
X_train_pad_train = np.expand_dims(X_train_pad_train, axis=-1)
X_train_pad_val = np.expand_dims(X_train_pad_val, axis=-1)
X_test_pad = np.expand_dims(X_test_pad, axis=-1)

# Define models without embedding layer
def build_rnn_model_no_embedding():
    model = Sequential()
    model.add(SimpleRNN(units=64, activation='relu', input_shape=(max_seq_length, 1)))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def build_gru_model_no_embedding():
    model = Sequential()
    model.add(GRU(units=64, activation='relu', input_shape=(max_seq_length, 1)))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def build_lstm_model_no_embedding():
    model = Sequential()
    model.add(LSTM(units=64, activation='relu', input_shape=(max_seq_length, 1)))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def build_bilstm_model_no_embedding():
    model = Sequential()
    model.add(Bidirectional(LSTM(units=64, activation='relu'), input_shape=(max_seq_length, 1)))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Train and evaluate models without embedding layer
models_no_embedding = {
    "RNN without embedding": build_rnn_model_no_embedding,
    "GRU without embedding": build_gru_model_no_embedding,
    "LSTM without embedding": build_lstm_model_no_embedding,
    "BiLSTM without embedding": build_bilstm_model_no_embedding
}

results_no_embedding = {}

# Train the model without embedding layer
for model_name, model_builder in models_no_embedding.items():
    model = model_builder()
    model.fit(X_train_pad_train, y_train_train, epochs=5, batch_size=64, validation_data=(X_train_pad_val, y_train_val), verbose=0)
    y_pred = (model.predict(X_test_pad) > 0.5).astype("int32")
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    results_no_embedding[model_name] = {
        "Accuracy": accuracy,
        "Precision": precision,
        "Recall": recall,
        "F1 Score": f1
    }

# Print results without embedding layer in a table
results_df_no_embedding = pd.DataFrame.from_dict(results_no_embedding, orient='index')
print(results_df_no_embedding)


c:\Users\Lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step


c:\Users\Lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step


c:\Users\Lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step


c:\Users\Lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step
                          Accuracy  Precision  Recall  F1 Score
RNN without embedding       0.5125   0.523540   0.278  0.363161
GRU without embedding       0.4985   0.498831   0.640  0.560666
LSTM without embedding      0.5015   0.502110   0.357  0.417300
BiLSTM without embedding    0.5000   0.000000   0.000  0.000000


c:\Users\Lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


question 2

Models with Embedding

In [ ]:

# Path to the dataset folder
dataset_path = "D:\\NLP\\data"

# Load data
def load_data(folder, limit):
    data = []
    labels = []
    for sentiment in ['neg', 'pos']:
        sentiment_folder = os.path.join(folder, sentiment)
        files = os.listdir(sentiment_folder)[:limit]
        for file in files:
            with open(os.path.join(sentiment_folder, file), 'r', encoding='utf-8') as f:
                data.append(f.read())
            labels.append(0 if sentiment == 'neg' else 1)
    return data, labels

# Limit the number of data samples
limit = 500
train_data, train_labels = load_data(os.path.join(dataset_path, 'train'), limit)
test_data, test_labels = load_data(os.path.join(dataset_path, 'test'), limit)

# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_data)
X_train = tokenizer.texts_to_sequences(train_data)
X_test = tokenizer.texts_to_sequences(test_data)

# Padding sequences
max_seq_length = 100
X_train_pad = pad_sequences(X_train, maxlen=max_seq_length)
X_test_pad = pad_sequences(X_test, maxlen=max_seq_length)

# Convert labels to binary (N -> 0, P -> 1)
y_train = train_labels
y_test = test_labels

# Define models
def build_rnn_model():
    model = Sequential()
    model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=100, input_length=max_seq_length))
    model.add(SimpleRNN(units=64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def build_gru_model():
    model = Sequential()
    model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=100, input_length=max_seq_length))
    model.add(GRU(units=64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def build_lstm_model():
    model = Sequential()
    model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=100, input_length=max_seq_length))
    model.add(LSTM(units=64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def build_bilstm_model():
    model = Sequential()
    model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=100, input_length=max_seq_length))
    model.add(Bidirectional(LSTM(units=64, activation='relu')))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Train and evaluate models
models = {
    "RNN": build_rnn_model,
    "GRU": build_gru_model,
    "LSTM": build_lstm_model,
    "BiLSTM": build_bilstm_model
}

results = {}

# Split the training data into training and validation sets
X_train_pad_train, X_train_pad_val, y_train_train, y_train_val = train_test_split(X_train_pad, y_train, test_size=0.2, random_state=42)

# Print the shape and content of the training and validation data
print("X_train_pad_train shape:", X_train_pad_train.shape)
print("X_train_pad_val shape:", X_train_pad_val.shape)
print("y_train_train shape:", len(y_train_train))
print("y_train_val shape:", len(y_train_val))

# Train the model
for model_name, model_builder in models.items():
    model = model_builder()
    model.fit(X_train_pad_train, y_train_train, epochs=5, batch_size=64, validation_data=(X_train_pad_val, y_train_val), verbose=0)
    y_pred = (model.predict(X_test_pad) > 0.5).astype("int32")
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    results[model_name] = {
        "Accuracy": accuracy,
        "Precision": precision,
        "Recall": recall,
        "F1 Score": f1
    }

# Print results in a table
results_df = pd.DataFrame.from_dict(results, orient='index')
print(results_df)


X_train_pad_train shape: (800, 100)
X_train_pad_val shape: (200, 100)
y_train_train shape: 800
y_train_val shape: 200
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step


c:\Users\Lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\embedding.py:86: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step


c:\Users\Lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\embedding.py:86: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step


c:\Users\Lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\embedding.py:86: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step
        Accuracy  Precision  Recall  F1 Score
RNN        0.579   0.593381   0.502  0.543879
GRU        0.598   0.558195   0.940  0.700447
LSTM       0.661   0.730659   0.510  0.600707
BiLSTM     0.675   0.648054   0.766  0.702108


**Question 3**

In [15]:
import tarfile
import urllib.request

In [16]:
url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
data_file = "aclImdb_v1.tar.gz"
urllib.request.urlretrieve(url, data_file)
tar = tarfile.open(data_file, "r:gz")
tar.extractall()
tar.close()

In [17]:
def load_data(folder, num_samples):
    data = []
    labels = []
    for sentiment in ["pos", "neg"]:
        sentiment_folder = os.path.join(folder, sentiment)
        for filename in os.listdir(sentiment_folder)[:num_samples//2]:
            with open(os.path.join(sentiment_folder, filename), "r", encoding="utf-8") as file:
                data.append(file.read())
            labels.append(1 if sentiment == "pos" else 0)
    return data, labels

In [18]:
folder = "aclImdb/train"
num_samples = 2500

train_data, train_labels = load_data(folder, num_samples)

print("Number of samples: ", len(train_data))
print("First 5 samples:")
for i in range(5):
    print(train_data[i])

print("\nCorresponding labels:")
print(train_labels[:5])

Number of samples:  2500
First 5 samples:
This movie is finally out on DVD in Italy (completely restored). I have seen this movie so many times and I find it even actual these days (2003) when Italy suffers again from a sort of brainwashing dictatorship (or the US for that matter). I am glad there are outcasts as the one played by Mastroianni in this movie who can sing out of tune; maybe they can teach the Sophia Lorens of this world how to be strong and fight to be recognised as human beings.<br /><br />Back to the movie: as most people here already mentioned the acting is wonderful but the audio background is astonishing. I must assume that unfortunately something is lost if you don't understand the Italian language but I can assure you that the show-off of machism, the distortion of reality in that ever-present radio-chronicle of the Hitler visit to Rome can really make you shiver!<br /><br />A masterpiece!<br /><br />
I haven't seen the original "Incredible Journey" since I was a c

In [19]:
folder = "aclImdb/test"
num_samples = 2500

test_data, test_labels = load_data(folder, num_samples)

print("Number of samples: ", len(test_data))
print("First 5 samples:")
for i in range(5):
    print(test_data[i])

print("\nCorresponding labels:")
print(test_labels[:5])

Number of samples:  2500
First 5 samples:
I was very curious about Anatomy (aka Anatomie) and if I was going to see it, I was going to have to buy it since no video stores in my area carried the film. Since it was not a low-priced DVD, I did take a chance and thought I'd take a peek at other comments on IMDb. Many of the comments didn't give me enough hope of forking out lots of bucks for a film I had never seen nor had any clues about. I basically got the idea it was a sexy youth-oriented romp being compared to many cookie-cutter teen thrillers. Well, something in the back of my mind told me to ignore those types of comments and buy it! I did, and was I pleasantly surprised!<br /><br />If it is going to be compared to any other films, I would say it's a variation of Coma and Extreme Measures. I couldn't see any comparison to films like Scream, Urban Legends, et al. Yes, the cast is young (that's because they're med students! At least they aren't the increasingly boring high school typ

In [21]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, SimpleRNN, GRU, LSTM, Bidirectional
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [23]:
train_data, val_data, train_labels, val_labels = train_test_split(train_data, train_labels, test_size=0.1, random_state=42)

# Vectorizing the text data into bag of words representation
vectorizer = CountVectorizer(max_features=2500)
train_features = vectorizer.fit_transform(train_data)
val_features = vectorizer.transform(val_data)
test_features = vectorizer.transform(test_data)
print(vectorizer)

CountVectorizer(max_features=2500)


In [24]:
maxlen = 150
train_features = pad_sequences(train_features.toarray(), maxlen=maxlen)
val_features = pad_sequences(val_features.toarray(), maxlen=maxlen)
test_features = pad_sequences(test_features.toarray(), maxlen=maxlen)

print("Shape of train_features: ", train_features.shape)
print("Shape of val_features: ", val_features.shape)
print("Shape of test_features: ", test_features.shape)

Shape of train_features:  (2250, 150)
Shape of val_features:  (250, 150)
Shape of test_features:  (2500, 150)


In [28]:
train_labels = np.array(train_labels)
val_labels = np.array(val_labels)
test_labels = np.array(test_labels)

train_labels = train_labels.reshape(-1, 1)
val_labels = val_labels.reshape(-1, 1)
test_labels = test_labels.reshape(-1, 1)

print("Shape of train_labels: ", train_labels.shape)
print("Shape of val_labels: ", val_labels.shape)
print("Shape of test_labels: ", test_labels.shape)

Shape of train_labels:  (2250, 1)
Shape of val_labels:  (250, 1)
Shape of test_labels:  (2500, 1)


Word2Vec

In [1]:
pip install gensim


In [4]:
pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.12.0-py3-none-any.whl (234 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4227143 sha256=9400645897dfe5484eb206dec1fcbbfbe7ca4b1d9c51d84cbb8e6c92a24b9988
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext


In [2]:
import gensim

Word2Vec

In [4]:
import gensim.downloader as api
from gensim.models import KeyedVectors
import fasttext

# Load Word2Vec model
w2v_model = api.load("word2vec-google-news-300")
word2vec_embeddings = w2v_model.vectors

# Get vocabulary size
word2vec_vocab_size = len(w2v_model.key_to_index)

[==================================================] 100.0% 1662.8/1662.8MB downloaded


Fast text

In [6]:
|import gensim.downloader as api

ft_model = api.load("fasttext-wiki-news-subwords-300")
fasttext_embeddings = ft_model.vectors
fasttext_vocab_size = len(ft_model.key_to_index)

print("FastText vocabulary size:", fasttext_vocab_size)

[==================================================] 100.0% 958.5/958.4MB downloaded
FastText vocabulary size: 999999


RNN

In [29]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
from scipy.sparse import issparse

# Function to sort the indices of a sparse matrix
def sort_sparse_indices(X):
    if issparse(X):
        X.sort_indices()

# Function to build RNN model with specified input shape
def build_rnn_model(input_shape):
    model = Sequential()
    model.add(Dense(64, input_shape=input_shape, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    return model

# Perform train-validation-test split
X_train, X_val, y_train, y_val = train_test_split(train_data, train_labels, test_size=0.1, random_state=42)
y_train = y_train.flatten()
y_val = y_val.flatten()
X_train = np.concatenate((X_train, X_val), axis=0)
y_train = np.concatenate((y_train, y_val), axis=0)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

# Build and train model with Bag-of-Words (BoW)
vectorizer = CountVectorizer(max_features=10000)
X_bow = vectorizer.fit_transform(X_train)

# Sort the indices of the BoW matrix
sort_sparse_indices(X_bow)

# Split the data into training and validation sets
X_train_bow, X_val_bow, y_train, y_val = train_test_split(X_bow, y_train, test_size=0.1, random_state=42)

# Define RNN model with BoW embeddings
input_shape = (X_train_bow.shape[1],)
rnn_model_bow = build_rnn_model(input_shape)

# Compile and train the RNN model with BoW embeddings
rnn_model_bow.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
rnn_model_bow.fit(X_train_bow, y_train, epochs=8, batch_size=32, validation_data=(X_val_bow, y_val))

# Transform the test data using the same vectorizer
X_test_bow = vectorizer.transform(X_test)

# Sort the indices of the BoW matrix
sort_sparse_indices(X_test_bow)

# Predict using the model
y_pred_rnn_bow = np.round(rnn_model_bow.predict(X_test_bow))
accuracy_rnn_bow = accuracy_score(y_test, y_pred_rnn_bow)
precision_rnn_bow = precision_score(y_test, y_pred_rnn_bow)
recall_rnn_bow = recall_score(y_test, y_pred_rnn_bow)
f1_score_rnn_bow = f1_score(y_test, y_pred_rnn_bow)

word2vec_input_shape = (100,)

# Build and train model with Word2Vec embeddings
rnn_model_w2v = build_rnn_model(word2vec_input_shape)
rnn_model_w2v.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
rnn_model_w2v.fit(X_train_seq, y_train, epochs=8, batch_size=32, validation_split=0.1)

# Evaluate model with Word2Vec embeddings
y_pred_rnn_w2v = np.round(rnn_model_w2v.predict(X_test_seq))
accuracy_rnn_w2v = accuracy_score(y_test, y_pred)
precision_rnn_w2v = precision_score(y_test, y_pred)
recall_rnn_w2v = recall_score(y_test, y_pred_rnn)
f1_score_rnn_w2v = f1_score(y_test, y_pred)

# Build and train model with FastText embeddings
rnn_model_ft = build_rnn_model(input_shape)
rnn_model_ft.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
rnn_model_ft.fit(y_test, y_train, epochs=8, batch_size=32, validation_split=0.1)

# Evaluate model with FastText embeddings
y_pred_rnn_ft = np.round(rnn_model_ft.predict(X_test_seq))
accuracy_rnn_ft = accuracy_score(y_test, y_pred_rnn_ft)
precision_rnn_ft = precision_score(y_test, y_pred_rnn_ft)
recall_rnn_ft = recall_score(y_test, y_pred_rnn_ft)
f1_score_rnn_ft = f1_score(y_test, y_pred_rnn_ft)

models = ['RNN with default Embedding','RNN with BoW', 'RNN with Word2Vec', 'RNN with FastText']
metrics = ['Accuracy', 'Precision', 'Recall', 'F1 Score']

data = [
    [accuracy,precision,recall,f1_score],
    [accuracy_rnn_bow, precision_rnn_bow, recall_rnn_bow, f1_score_rnn_bow],
    [accuracy_rnn_w2v, precision_rnn_w2v, recall_rnn_w2v, f1_score_rnn_w2v],
    [accuracy_rnn_ft, precision_rnn_ft, recall_rnn_ft, f1_score_rnn_ft]
]

print('| Model | Accuracy | Precision | Recall | F1 Score |')
print('|-------|----------|-----------|--------|----------|')
for i in range(len(models)):
    model = models[i]
    accuracy = data[i][0]
    precision = data[i][1]
    recall = data[i][2]
    f1_score = data[i][3]
    print(f'| {model} | {accuracy:.4f} | {precision:.4f} | {recall:.4f} | {f1_score:.4f} |')

Epoch 1/8
48/48 [==============================] - 2s 14ms/step - loss: 0.5603 - accuracy: 0.7306 - val_loss: 0.4892 - val_accuracy: 0.7811
Epoch 2/8
48/48 [==============================] - 1s 11ms/step - loss: 0.2190 - accuracy: 0.9519 - val_loss: 0.3908 - val_accuracy: 0.8462
Epoch 3/8
48/48 [==============================] - 1s 11ms/step - loss: 0.0899 - accuracy: 0.9928 - val_loss: 0.3614 - val_accuracy: 0.8639
Epoch 4/8
48/48 [==============================] - 1s 11ms/step - loss: 0.0369 - accuracy: 0.9993 - val_loss: 0.3554 - val_accuracy: 0.8580
Epoch 5/8
48/48 [==============================] - 0s 10ms/step - loss: 0.0186 - accuracy: 0.9993 - val_loss: 0.3611 - val_accuracy: 0.8639
Epoch 6/8
48/48 [==============================] - 1s 11ms/step - loss: 0.0106 - accuracy: 1.0000 - val_loss: 0.3685 - val_accuracy: 0.8698
Epoch 7/8
48/48 [==============================] - 0s 10ms/step - loss: 0.0065 - accuracy: 1.0000 - val_loss: 0.3820 - val_accuracy: 0.8817
Epoch 8/8
18/18 [===

NameError: name 'X_train_seq' is not defined

GRU

In [30]:
from sklearn.feature_extraction.text import CountVectorizer

# Function to build GRU model with specified embedding matrix
def build_gru_with_pretrained_embedding(embedding_matrix):
    model = Sequential()
    model.add(Embedding(input_dim=embedding_matrix.shape[0], output_dim=embedding_matrix.shape[1],
                        weights=[embedding_matrix], input_length=maxlen, trainable=False))
    model.add(GRU(64))
    model.add(Dense(1, activation='sigmoid'))
    return model

# Function to build GRU model with BoW embedding
def build_gru_with_bow_embedding():
    model = Sequential()
    model.add(Dense(64, input_shape=(X_train_bow.shape[1],), activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    return model

# Build and train model with Bag-of-Words (BoW) embedding
vectorizer = CountVectorizer(max_features=10000)
X_train_bow = vectorizer.fit_transform(X_train)
X_test_bow = vectorizer.transform(X_test)

gru_model_bow = build_gru_with_bow_embedding()
gru_model_bow.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
gru_model_bow.fit(X_train_bow, y_train, epochs=8, batch_size=32, validation_split=0.1)

# Evaluate model with BoW embeddings
y_pred_gru_bow = np.round(gru_model_bow.predict(X_test_bow))
accuracy_gru_bow = accuracy_score(y_test, y_pred_gru_bow)
precision_gru_bow = precision_score(y_test, y_pred_gru_bow)
recall_gru_bow = recall_score(y_test, y_pred_gru_bow)
f1_score_gru_bow = f1_score(y_test, y_pred_gru_bow)

# Build and train model with Word2Vec embeddings
gru_model_w2v = build_gru_with_pretrained_embedding(word2vec_embeddings)
gru_model_w2v.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
gru_model_w2v.fit(X_train_seq, y_train, epochs=8, batch_size=32, validation_split=0.1)

# Evaluate model with Word2Vec embeddings
y_pred_gru_w2v = np.round(gru_model_w2v.predict(X_test_seq))
accuracy_gru_w2v = accuracy_score(y_test, y_pred_gru_w2v)
precision_gru_w2v = precision_score(y_test, y_pred_gru_w2v)
recall_gru_w2v = recall_score(y_test, y_pred_gru_w2v)
f1_score_gru_w2v = f1_score(y_test, y_pred_gru_w2v)

# Build and train model with FastText embeddings
gru_model_ft = build_gru_with_pretrained_embedding(fasttext_embeddings)
gru_model_ft.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
gru_model_ft.fit(X_train_seq, y_train, epochs=8, batch_size=32, validation_split=0.1)

# Evaluate model with FastText embeddings
y_pred_gru_ft = np.round(gru_model_ft.predict(X_test_seq))
accuracy_gru_ft = accuracy_score(y_test, y_pred_gru_ft)
precision_gru_ft = precision_score(y_test, y_pred_gru_ft)
recall_gru_ft = recall_score(y_test, y_pred_gru_ft)
f1_score_gru_ft = f1_score(y_test, y_pred_gru_ft)

models = ['GRU with default Embedding','GRU with BoW', 'GRU with Word2Vec', 'GRU with FastText']
metrics = ['Accuracy', 'Precision', 'Recall', 'F1 Score']

data = [
    [accuracy,precision,recall,f1_score],
    [accuracy_gru_bow, precision_gru_bow, recall_gru_bow, f1_score_gru_bow],
    [accuracy_gru_w2v, precision_gru_w2v, recall_gru_w2v, f1_score_gru_w2v],
    [accuracy_gru_ft, precision_gru_ft, recall_gru_ft, f1_score_gru_ft]
]

print('| Model | Accuracy | Precision | Recall | F1 Score |')
for i in range(len(models)):
    model = models[i]
    accuracy = data[i][0]
    precision = data[i][1]
    recall = data[i][2]
    f1_score = data[i][3]
    print(f'| {model} | {accuracy:.4f} | {precision:.4f} | {recall:.4f} | {f1_score:.4f} |')

ValueError: `validation_split` is only supported for Tensors or NumPy arrays, found following types in the input: [<class 'scipy.sparse._csr.csr_matrix'>]

LSTM

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Function to build LSTM model with specified embedding matrix
def build_lstm_with_pretrained_embedding(embedding_matrix):
    model = Sequential()
    model.add(Embedding(input_dim=embedding_matrix.shape[0], output_dim=embedding_matrix.shape[1],
                        weights=[embedding_matrix], input_length=maxlen, trainable=False))
    model.add(LSTM(64))
    model.add(Dense(1, activation='sigmoid'))
    return model

# Function to build LSTM model with BoW embedding
def build_lstm_with_bow_embedding():
    model = Sequential()
    model.add(Dense(64, input_shape=(X_train_bow.shape[1],), activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    return model

# Build and train model with Bag-of-Words (BoW) embedding
vectorizer = CountVectorizer(max_features=10000)
X_train_bow = vectorizer.fit_transform(X_train)
X_test_bow = vectorizer.transform(X_test)

lstm_model_bow = build_lstm_with_bow_embedding()
lstm_model_bow.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
lstm_model_bow.fit(X_train_bow, y_train, epochs=8, batch_size=32, validation_split=0.1)

# Evaluate model with BoW embeddings
y_pred_lstm_bow = np.round(lstm_model_bow.predict(X_test_bow))
accuracy_lstm_bow = accuracy_score(y_test, y_pred_lstm_bow)
precision_lstm_bow = precision_score(y_test, y_pred_lstm_bow)
recall_lstm_bow = recall_score(y_test, y_pred_lstm_bow)
f1_score_lstm_bow = f1_score(y_test, y_pred_lstm_bow)

# Build and train model with Word2Vec embeddings
lstm_model_w2v = build_lstm_with_pretrained_embedding(word2vec_embeddings)
lstm_model_w2v.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
lstm_model_w2v.fit(X_train_seq, y_train, epochs=8, batch_size=32, validation_split=0.1)

# Evaluate model with Word2Vec embeddings
y_pred_lstm_w2v = np.round(lstm_model_w2v.predict(X_test_seq))
accuracy_lstm_w2v = accuracy_score(y_test, y_pred_lstm_w2v)
precision_lstm_w2v = precision_score(y_test, y_pred_lstm_w2v)
recall_lstm_w2v = recall_score(y_test, y_pred_lstm_w2v)
f1_score_lstm_w2v = f1_score(y_test, y_pred_lstm_w2v)

# Build and train model with FastText embeddings
lstm_model_ft = build_lstm_with_pretrained_embedding(fasttext_embeddings)
lstm_model_ft.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
lstm_model_ft.fit(X_train_seq, y_train, epochs=8, batch_size=32, validation_split=0.1)

# Evaluate model with FastText embeddings
y_pred_lstm_ft = np.round(lstm_model_ft.predict(X_test_seq))
accuracy_lstm_ft = accuracy_score(y_test, y_pred_lstm_ft)
precision_lstm_ft = precision_score(y_test, y_pred_lstm_ft)
recall_lstm_ft = recall_score(y_test, y_pred_lstm_ft)
f1_score_lstm_ft = f1_score(y_test, y_pred_lstm_ft)

models = ['LSTM with default Embedding','LSTM with BoW', 'LSTM with Word2Vec', 'LSTM with FastText']
metrics = ['Accuracy', 'Precision', 'Recall', 'F1 Score']

data = [
    [accuracy,precision,recall,f1_score],
    [accuracy_lstm_bow, precision_lstm_bow, recall_lstm_bow, f1_score_lstm_bow],
    [accuracy_lstm_w2v, precision_lstm_w2v, recall_lstm_w2v, f1_score_lstm_w2v],
    [accuracy_lstm_ft, precision_lstm_ft, recall_lstm_ft, f1_score_lstm_ft]
]

print('| Model | Accuracy | Precision | Recall | F1 Score |')
for i in range(len(models)):
    model = models[i]
    accuracy = data[i][0]
    precision = data[i][1]
    recall = data[i][2]
    f1_score = data[i][3]
    print(f'| {model} | {accuracy:.4f} | {precision:.4f} | {recall:.4f} | {f1_score:.4f} |')

BiLSTM

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Function to build BiLSTM model with specified embedding matrix
def build_bilstm_with_pretrained_embedding(embedding_matrix):
    model = Sequential()
    model.add(Embedding(input_dim=embedding_matrix.shape[0], output_dim=embedding_matrix.shape[1],
                        weights=[embedding_matrix], input_length=maxlen, trainable=False))
    model.add(Bidirectional(LSTM(64)))
    model.add(Dense(1, activation='sigmoid'))
    return model

# Function to build BiLSTM model with BoW embedding
def build_bilstm_with_bow_embedding():
    model = Sequential()
    model.add(Dense(64, input_shape=(X_train_bow.shape[1],), activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    return model

# Build and train model with Bag-of-Words (BoW) embedding
vectorizer = CountVectorizer(max_features=10000)
X_train_bow = vectorizer.fit_transform(X_train)
X_test_bow = vectorizer.transform(X_test)

bilstm_model_bow = build_bilstm_with_bow_embedding()
bilstm_model_bow.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
bilstm_model_bow.fit(X_train_bow, y_train, epochs=8, batch_size=32, validation_split=0.1)

# Evaluate model with BoW embeddings
y_pred_bilstm_bow = np.round(bilstm_model_bow.predict(X_test_bow))
accuracy_bilstm_bow = accuracy_score(y_test, y_pred_bilstm_bow)
precision_bilstm_bow = precision_score(y_test, y_pred_bilstm_bow)
recall_bilstm_bow = recall_score(y_test, y_pred_bilstm_bow)
f1_score_bilstm_bow = f1_score(y_test, y_pred_bilstm_bow)

# Build and train model with Word2Vec embeddings
bilstm_model_w2v = build_bilstm_with_pretrained_embedding(word2vec_embeddings)
bilstm_model_w2v.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
bilstm_model_w2v.fit(X_train_seq, y_train, epochs=8, batch_size=32, validation_split=0.1)

# Evaluate model with Word2Vec embeddings
y_pred_bilstm_w2v = np.round(bilstm_model_w2v.predict(X_test_seq))
accuracy_bilstm_w2v = accuracy_score(y_test, y_pred_bilstm_w2v)
precision_bilstm_w2v = precision_score(y_test, y_pred_bilstm_w2v)
recall_bilstm_w2v = recall_score(y_test, y_pred_bilstm_w2v)
f1_score_bilstm_w2v = f1_score(y_test, y_pred_bilstm_w2v)

# Build and train model with FastText embeddings
bilstm_model_ft = build_bilstm_with_pretrained_embedding(fasttext_embeddings)
bilstm_model_ft.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
bilstm_model_ft.fit(X_train_seq, y_train, epochs=8, batch_size=32, validation_split=0.1)

# Evaluate model with FastText embeddings
y_pred_bilstm_ft = np.round(bilstm_model_ft.predict(X_test_seq))
accuracy_bilstm_ft = accuracy_score(y_test, y_pred_bilstm_ft)
precision_bilstm_ft = precision_score(y_test, y_pred_bilstm_ft)
recall_bilstm_ft = recall_score(y_test, y_pred_bilstm_ft)
f1_score_bilstm_ft = f1_score(y_test, y_pred_bilstm_ft)

models = ['BiLSTM with default Embedding','BiLSTM with BoW', 'BiLSTM with Word2Vec', 'BiLSTM with FastText']
metrics = ['Accuracy', 'Precision', 'Recall', 'F1 Score']

data = [
    [accuracy,precision,recall,f1_score],
    [accuracy_bilstm_bow, precision_bilstm_bow, recall_bilstm_bow, f1_score_bilstm_bow],
    [accuracy_bilstm_w2v, precision_bilstm_w2v, recall_bilstm_w2v, f1_score_bilstm_w2v],
    [accuracy_bilstm_ft, precision_bilstm_ft, recall_bilstm_ft, f1_score_bilstm_ft]
]

print('| Model | Accuracy | Precision | Recall | F1 Score |')
for i in range(len(models)):
    model = models[i]
    accuracy = data[i][0]
    precision = data[i][1]
    recall = data[i][2]
    f1_score = data[i][3]
    print(f'| {model} | {accuracy:.4f} | {precision:.4f} | {recall:.4f} | {f1_score:.4f} |')